##                                 有关幻影坦克技术的讲解
啥是幻影坦克? 幻影坦克就是, 一张黑白图片, 在黑色背景下和白色背景下能够显示出不同的图像.

它的原理就是控制像素的颜色和Alpha通道(不透明度), 来使显示的图像在不同背景下显示不同的颜色.

最基本的, 就是, 一张半透明的黑色薄膜, 如果在黑色的纸上, 你什么也看不出来, 但如果在白色纸上, 你可以看见, 它是灰色.

先统一将Alpha值和像素的亮度统一至 0 - 1

###                                         基本原理
设: 这个图像的背景亮度为 bc, 这个像素的亮度为 pc, 不透明度为 pa, 则最终显示的颜色 oc 就是:
                           oc = pc * pa + bc * (1 - pa)

理解起来也简单, 还拿刚刚的例子来讲, 例如一个纯黑的半透明薄膜, 那他肯定:

1.只有一半的黑色能显示出来, 即： pc * pa

2.而背景色, 也有一半的颜色能够透过来. 即:   bc * (1 - pa)

3.总的颜色加起来, 也就是： pc * pa + bc * (1 - pa)

###                                         幻影坦克
当一个像素为白色背景时, 能够显示出一个特定的颜色 x, 当黑色背景时, 显示出 y, 故可得出一个公式：

  设颜色 x 的亮度为 xc, 颜色 y 的亮度为 yc, 这个像素的亮度为 zc, 不透明度为 za, 则满足:
 
$$
\begin{cases}
& \text{xc = za × zc + (1 − za)}\\
& \text{yc = za × zc}
\end{cases}
$$

稍微处理一下，则可以得到以下的公式：

$$
\begin{cases}
& \text{xc = yc + 1 × ( 1 − za )}\\
& \text{yc = xc − 1 × ( 1 − za)}\\
& \text{za = − [( xc − yc ) ÷ 1 ] + 1}
\end{cases}
$$

$$
\begin{cases}
& \text{xc = yc + 1 − za}\\
& \text{yc = xc + za − 1}\\
& \text{za = yc − xc + 1}
\end{cases}
$$
 



由于 xc, yc, zc, za 都是小于等于1, 大于等于0的值, 所以:

$$
\begin{cases}
& \text{xc = za × zc + (1 − za)}\\
& \text{yc = za × zc}
\end{cases}
$$

所以我们最终的算式为：

$$
\begin{cases}
& \text{xc = za × zc + (1 − za)}\\
& \text{yc = za × zc}
\end{cases}
$$

由于ARGB的通道是0 - 255 ，所以我们需要将以进行转化：

$$
\begin{cases}
& \text{xc = za × zc + (1 − za)}\\
& \text{yc = za × zc}
\end{cases}
$$

而经过刚才的推导，我们真正需要处理的只有：

$$ xc ≥ yc $$

## 代码实现

我们采用opencv库来对图像进行处理

首先我们可以定义一个函数对 xc ，yc 进行处理，使 xc ≥ yc ：

### 定义函数使 xc ≥ yc 

In [6]:
import numpy as np

In [7]:
def xc_max_than_yc(xc, yc, color_ratio=0.5):
    
    threshold = 255 * color_ratio
    xc = (xc / 255) * (255 - threshold) + threshold
    yc = (yc / 255) * threshold
    return xc, yc 

In [8]:
xc = np.random.randint(255,size = (5,5))
yc = np.random.randint(255,size = (5,5))
xc_,yc_ = xc_max_than_yc(xc,yc)
xc,yc,xc_,yc_

(array([[180, 155, 128, 199, 168],
        [ 20,  52,   5, 202, 145],
        [183, 174, 101,  58, 242],
        [ 97, 163, 229, 146,  40],
        [100,  46, 178, 187,  91]]),
 array([[188,  96, 112, 109,  69],
        [ 10, 164,  29, 212,  70],
        [116, 144, 167, 188,  53],
        [101, 177, 153, 245,  42],
        [108,  80,  16, 167,   1]]),
 array([[217.5, 205. , 191.5, 227. , 211.5],
        [137.5, 153.5, 130. , 228.5, 200. ],
        [219. , 214.5, 178. , 156.5, 248.5],
        [176. , 209. , 242. , 200.5, 147.5],
        [177.5, 150.5, 216.5, 221. , 173. ]]),
 array([[ 94. ,  48. ,  56. ,  54.5,  34.5],
        [  5. ,  82. ,  14.5, 106. ,  35. ],
        [ 58. ,  72. ,  83.5,  94. ,  26.5],
        [ 50.5,  88.5,  76.5, 122.5,  21. ],
        [ 54. ,  40. ,   8. ,  83.5,   0.5]]))

上述函数对xc,yc进行处理，color_ratio参数用于调整xc，yc的颜色占比

其中xc，yc输入的都是图像的像素矩阵

### 效果图生成函数

In [28]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
def create_tank(white, black):
    # white表示白色背景时显示的图片
    # black表示黑色背景时显示的图片
    # 接下来对 透明度（za） 和 亮度（zc）进行处理
    za = black - white + 255
    zc = black.copy()
    idx = black != 0
    zc[idx] = black[idx] * 255 / za[idx]
    img = cv2.merge((zc, zc, zc, za))
    return img


In [29]:
img = create_tank(xc_,yc_)
img

array([[[182.28136882, 182.28136882, 182.28136882, 131.5       ],
        [124.89795918, 124.89795918, 124.89795918,  98.        ],
        [119.49790795, 119.49790795, 119.49790795, 119.5       ],
        [168.45454545, 168.45454545, 168.45454545,  82.5       ],
        [112.78846154, 112.78846154, 112.78846154,  78.        ]],

       [[ 10.40816327,  10.40816327,  10.40816327, 122.5       ],
        [113.95095368, 113.95095368, 113.95095368, 183.5       ],
        [ 26.50537634,  26.50537634,  26.50537634, 139.5       ],
        [204.        , 204.        , 204.        , 132.5       ],
        [ 99.16666667,  99.16666667,  99.16666667,  90.        ]],

       [[157.34042553, 157.34042553, 157.34042553,  94.        ],
        [163.2       , 163.2       , 163.2       , 112.5       ],
        [132.6635514 , 132.6635514 , 132.6635514 , 160.5       ],
        [124.51948052, 124.51948052, 124.51948052, 192.5       ],
        [204.77272727, 204.77272727, 204.77272727,  33.        ]],

    

### 接下来则是以具体的图片举例


In [37]:
img_false = cv2.imread('img1.jpg')
img_true = cv2.imread('img2.jpg')
x,y,z = img_true.shape
img_false = cv2.resize(img_false, (y, x))
xc, yc = xc_max_than_yc(img_false, img_true)
Img_output = create_tank(xc, yc)
cv2.imwrite("Img_output.png", Img_output)

In [39]:
img = cv2.imread('Img_output')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.imshow(img)
plt.show()

error: OpenCV(4.7.0) d:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:94: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0x981fb336::Set<3,4,-1>,struct cv::impl::A0x981fb336::Set<0,2,5>,2>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Unsupported depth of input image:
>     'VDepth::contains(depth)'
> where
>     'depth' is 6 (CV_64F)
